### Preprocess

In [1]:
%%capture
!pip install -r requirements.txt --user

In [1]:
path_to_MyModule = '..'

import sys
sys.path.insert(0, path_to_MyModule) 

import pandas as pd

from MyModule.GeneralFunctions import *
from MyModule.SummarizationFunctions import *
from MyModule.SamplingFunctions import *

In [2]:
df = pd.read_excel('..\datos.xlsx')[['ID','texto','desafio']]

In [3]:
# Limpieza
df.drop_duplicates(subset='texto', inplace=True)

# Quitando texto de mas en columna "desafio"
df['desafio'] = df['desafio'].apply(lambda x: re.findall('[0-9]+', x)[0])

# A str
df['texto'] = df['texto'].astype(str)

# Summary of each topic

Abstractive summary with T5

In [19]:
df[df['desafio']=='15']['texto']

0                                                  False
5      Concuerdo parcialmente, dependiendo de en que ...
10     Creo que hay una idea errónea de los planes so...
15     Si bien varios estudios han demostrado que dar...
25     Creo que el tema de los planes sociales genera...
                             ...                        
549    Está bien el tema de los planes... pero hay qu...
552    En líneas generales estoy de acuerdo. Creo que...
554    El Estado es el principal responsable de que h...
556    Pienso que a mis abuelos ni se les ocurría ped...
558                                    Estoy de acuerdo.
Name: texto, Length: 83, dtype: object

In [11]:
t5 = T5Summary()

desafios = df['desafio'].unique()
for des in desafios:
    text = '\n'.join(df[df['desafio']==des]['texto'])
    print(f'Desafio {des}:\n{t5.generate_summary(text, num_beams=10)}\n\n')

ckpt:
josmunpen/mt5-small-spanish-summarization
Desafio 15:
['el estado quiere ayudar a garantizar el derecho a la alimentación y la salud', 'el estado debe ayudar a garantizar el derecho a la alimentación y la salud']


Desafio 13:
['partidos de izquierda "me siento mas identificado con los partidos de izquierda"', 'partidos de izquierda luchan por una distribución de la riqueza más justa']


Desafio 14:
['a ninguna persona, ninguna figura, ninguna figura que me genera esas emociones', 'ella de carloto "no tengo referente de la política argentina"']


Desafio 12:
['la izquierda moderada', 'la izquierda moderada, una persona de izquierda moderada']


Desafio 16:
['los subsidios a las empresas de transportes no son las mismas que cuando fundaron', 'los subsidios a las empresas de transportes no son las mismas que cuando fundaron']


Desafio 17:
['la hora de cursada sea menor', 'la hora de cursada sea menor']


Desafio 18:
['los horarios de cursada', 'los horarios de cursada']




# Select the most representative documents of each topic

In [13]:
from MyModule.SummarizationFunctions import MostRepresentativeDocs

pp_object = Preprocess(lemma=False)
mrd = MostRepresentativeDocs()

In [14]:
all_res = {}
for des in df['desafio'].unique():
    data = df[df['desafio']==des]['texto'].values
    result = mrd.get_representatives(data, 5, pp_object)
    all_res[des] = result

In [137]:
all_res['12'][3]

[('Soy peronista y k. Peronista x historia familiar, del pais y xq estoy convencida que el camino es x medio de la justicia social, los derechos laborales, la regulación del estado de y la distribución de la riqueza donde los mayores privilegiados sean los niños que sin el futuro del país. Y k xq soy contemporáneo y estoy convencida que hicieron  bien al país y siguieran las doctrinas peronistas',
  0.76536745),
 ('Nacionalista y popular. Porque considero que la economía se debe manejar en función de los intereses de las mayorías, las clases populares y los intereses nacionales, en defensa de la patria y sus recursos, y defender un mercado interno fuerte con buenos salarios ',
  0.72683716),
 ('Probablemente no podría. Entiendo que lo más cercano sería decir que valoro el desarrollo de capacidades en el estado que permitan generar políticas públicas de calidad para mejorar la situación del país y sus habitantes. Asimismo, creo que el intercambio de ideas es una de las cosas que nos hac

# Sample representative docs

In [5]:
desafio = '12'

documents = df[df['desafio']==desafio]['texto'].values

pp_object = Preprocess(lemma=False, stopwords=False, alphanumeric=False, join=False)

sr = SampleRepresentatives()
res = sr.get_sample(documents, pp_object=pp_object, n_clusters=5)

In [6]:
print(f'Desafio: {desafio}')
for key, value in res.items():
    print(f'\nCluster: {key}\n')
    for v in value[:3]:
        print(v)

Desafio: 12

Cluster: 2

No tengo una posición muy formada, pero me identifico con el liberalismo. Completamente liberal en lo social, y relativamente en lo económico. 
Creo que las narrativas en términos politicos y económicos actuales no están a la altura del mundo actual. Son ideologías viejas surgidas en contextos totalmente diferentes, no responden a las necesidades del mundo actual, sus problemáticas y sus avances. Estamos viviendo el inicio de la cuarta revolución industrial y tenemos herramientas sociales de la era de los motores a vapor. Pero respondiendo a la consigna; creo que en términos económicos me siento mas identificado con el libre flujo de capitales y con control activo de un BUEN estado. 
Me defino como kirchnerista. Soberania política, independencia económica y justocia social.

Cluster: 0

Nacionalista y popular. Porque considero que la economía se debe manejar en función de los intereses de las mayorías, las clases populares y los intereses nacionales, en defensa

# Combination: summarization for each topic's cluster

Vectorice each doc

In [31]:
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [32]:
df['desafio'].unique()

array(['15', '13', '14', '12', '16', '17', '18'], dtype=object)

In [67]:
desafio = '14'

original_documents = df.query(f'desafio == "{desafio}"')['texto'].values
documents = pp_object.preprocess(original_documents)

emb_docs = model.encode(original_documents)

Cluster them with K-Means

In [68]:
n_clusters = 5
kmeans = KMeans(
            init="k-means++",
            n_clusters=n_clusters,
            n_init=10,
            max_iter=300)

In [69]:
kmeans.fit(emb_docs)

KMeans(n_clusters=5, n_init=10)

In [70]:
labeled_docs = {}
for i, orig_doc in enumerate(original_documents):
    current_label = kmeans.labels_[i]
    if current_label in labeled_docs.keys():
        labeled_docs[current_label].append(orig_doc)
    else:
        labeled_docs[current_label] = [orig_doc]

Apply T5 and BETO summarization to each cluster

In [73]:
from time import sleep

beto = BETOSummary()

t5 = T5Summary()

for key, values in labeled_docs.items():
    text = '.\n'.join(values)
    print(f'BERT, Cluster {key}:\n{beto.generate_summary(text)}\n\n')
    sleep(2)
    print(f'T5, Cluster {key}:\n{t5.generate_summary(text)}\n\n')

ckpt:
mrm8488/bert2bert_shared-spanish-finetuned-summarization


The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


ckpt:
josmunpen/mt5-small-spanish-summarization
BERT, Cluster 4:
['Muy difícil de responder. Puedo pensar solo en referentes científicos y deportistas. A ninguna, ninguna figura relativamente conocida me genera esas emociones... No tengo preferencias por ninguna persona', 'A ninguna, ninguna figura relativamente conocida me genera esas emociones. No tengo preferencias por ninguna persona en particular. No sé. Hoy nadie']


T5, Cluster 4:
['"no tengo a alguna persona para mencionar a ninguna persona en particular"', '"no tengo a alguna persona para mencionar a ninguna persona en particular"']


BERT, Cluster 2:
['La verdad nose, la unica persona que mas relaciones positivas a nivel internacional genera, y la wur mas representa los intereses de la ckade trabajadora', 'La verdad nose, la unica persona más influyente politicamente, la que mas relaciones positivas a nivel internacional genera, y la wur mas representa los intereses de la ckade trabajadora']


T5, Cluster 2:
['sánchez, el act